<a href="https://colab.research.google.com/github/Sun-Deep/mask-rcnn-opencv/blob/master/mask_rcnn_opencv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Sun-Deep/mask-rcnn-opencv.git

Cloning into 'mask-rcnn-opencv'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 1), reused 13 (delta 1), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [2]:
cd mask-rcnn-opencv/

/content/mask-rcnn-opencv


In [3]:
!pwd

/content/mask-rcnn-opencv


In [4]:
!ls

images	mask-rcnn-coco	mask_rcnn.py  mask_rcnn_video.py


In [6]:
!pip install virtualenv

     |████████████████████████████████| 2.0MB 3.4MB/s 


In [7]:
!ls

images	mask-rcnn-coco	mask_rcnn.py  mask_rcnn_video.py


In [8]:
!virtualenv env

Using base prefix '/usr'
New python executable in /content/mask-rcnn-opencv/env/bin/python3
Also creating executable in /content/mask-rcnn-opencv/env/bin/python
Installing setuptools, pip, wheel...
done.


In [9]:
!ls

env  images  mask-rcnn-coco  mask_rcnn.py  mask_rcnn_video.py


In [10]:
cd env

/content/mask-rcnn-opencv/env


In [11]:
ls

bin/  include/  lib/


In [12]:
cd ..

/content/mask-rcnn-opencv


In [0]:
!source env/bin/activate


In [18]:
!pip install numpy scipy

In [19]:
!pip install pillow scikit-image matplotlib imutils

In [20]:
!pip install "IPython[all]"

     |████████████████████████████████| 204kB 3.1MB/s 


In [21]:
!pip install tensorflow-gpu

     |████████████████████████████████| 377.0MB 48kB/s 
     |████████████████████████████████| 3.2MB 36.1MB/s 
ERROR: tensorflow 1.14.0rc1 has requirement tensorboard<1.14.0,>=1.13.0, but you'll have tensorboard 1.14.0 which is incompatible.
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1


In [22]:
!pip install keras h5py

In [23]:
!pip install opencv-contrib-python

In [0]:
# import the necessary packages
import numpy as np
import argparse
import random
import time
import cv2
import os
import sys
import datetime
import imageio
from PIL import Image
from google.colab.patches import cv2_imshow

In [0]:
#construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True, help="path to input image")
ap.add_argument("-m", "--mask_rcnn", required=True, help="base path to mask rcnn directory")
ap.add_argument("-v", "--visualize", type=int, default=0, help="whether or not we are going to visualize each instance")
ap.add_argument("-c", "--confidence", type=float, default=0.5, help="minimum probability to filter weak detections")
ap.add_argument("-t", "--threshold", type=float, default=0.3, help="minimum threshold for pixel-wise mask segmentation")
args = {
	"image": "images/example_01.jpg",
	"output": "images/example_01_result.jpg",
  "mask_rcnn": "mask-rcnn-coco/object_detection_classes_coco.txt"
}

In [0]:
#load coco class labels our mask rcnn was trained on
LABELS = open('mask-rcnn-coco/object_detection_classes_coco.txt').read().strip().split('\n')

In [0]:
#load the set of the colors that will be used when visualizing a given instance segmentation
COLORS = open('mask-rcnn-coco/colors.txt').read().strip().split('\n')
COLORS = [np.array(c.split(",")).astype('int') for c in COLORS]
COLORS = np.array(COLORS, dtype='uint8')

In [0]:
#path to weight
weightsPath = 'mask-rcnn-coco/frozen_inference_graph.pb'
#path to model configuration
configPath = 'mask-rcnn-coco/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt'

#load our mask rcnn model trained on the coco dataset
net = cv2.dnn.readNetFromTensorflow(weightsPath, configPath)

In [0]:
image = cv2.imread('images/example_02.jpg')
(H, W) = image.shape[:2]

In [0]:
blob = cv2.dnn.blobFromImage(image, swapRB = True, crop = False)
net.setInput(blob)
start = time.time()
(boxes, masks) = net.forward(["detection_out_final", "detection_masks"])
end = time.time()

In [86]:
print("[INFO] Mask R-CNN took {:.6f} seconds".format(end - start))
print("[INFO] boxes shape: {}".format(boxes.shape))
print("[INFO] masks shape: {}".format(masks.shape))

[INFO] Mask R-CNN took 3.919642 seconds
[INFO] boxes shape: (1, 1, 8, 7)
[INFO] masks shape: (100, 90, 15, 15)


In [0]:
results = []
#loop over the number of detected objects
for i in range(0, boxes.shape[2]):
  #extract the class ID of the detection along with the confidence
  classID = int(boxes[0, 0, i, 1])
  confidence = boxes[0, 0, i, 2]
  
  #filter out the weak predictions by ensuring the detected probability is greater than minimum probability
  if confidence > 0.5:
    #clone our original image so we can draw on it
    clone = image.copy()
    
    #scale the bounding box coordinates back relative to the size of the image and then compute the width and the height of the bounding box
    box = boxes[0, 0, i, 3:7] * np.array([W, H, W, H])
    (startX, startY, endX, endY) = box.astype("int")
    boxW = endX - startX
    boxH = endY - startY
    
    # extract the pixel-wise segmentation for the object, resize
		# the mask such that it's the same dimensions of the bounding
		# box, and then finally threshold to create a *binary* mask
    mask = masks[i, classID]
    mask = cv2.resize(mask, (boxW, boxH), interpolation=cv2.INTER_NEAREST)
    mask = (mask > 0.3)
 
		# extract the ROI of the image
    roi = clone[startY:endY, startX:endX]
    
    # check to see if are going to visualize how to extract the
		# masked region itself
    if 0 > 0:
		  # convert the mask from a boolean to an integer mask with
			# to values: 0 or 255, then apply the mask
      visMask = (mask * 255).astype("uint8")
      instance = cv2.bitwise_and(roi, roi, mask=visMask)
 
			# show the extracted ROI, the mask, along with the
			# segmented instance
      cv2_imshow(roi)
      cv2_imshow(visMask)
      cv2_imshow(instance)
      
    # now, extract *only* the masked region of the ROI by passing
    # in the boolean mask array as our slice condition
    roi = roi[mask]
    # randomly select a color that will be used to visualize this
    # particular instance segmentation then create a transparent
    # overlay by blending the randomly selected color with the ROI
    color = random.choice(COLORS)
    blended = ((0.4 * color) + (0.6 * roi)).astype("uint8")

    # store the blended ROI in the original image
    clone[startY:endY, startX:endX][mask] = blended
      
    # draw the bounding box of the instance on the image
    color = [int(c) for c in color]
    cv2.rectangle(clone, (startX, startY), (endX, endY), color, 2)

    # draw the predicted label and associated probability of the
    # instance segmentation on the image
    text = "{}: {:.4f}".format(LABELS[classID], confidence)
    cv2.putText(clone, text, (startX, startY - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    results.append(clone)


In [0]:
# Open all the frames
images = []

for n in results:
    frame = Image.fromarray(n)
    images.append(frame)

# Save the frames as an animated GIF
images[0].save('output.gif',
               save_all=True,
               append_images=images[1:],
               duration=1000,
               loop=0)